# Inference

A module for interacting with integrated gradients

In [ ]:
from models import MODEL_DIR, load_model
from data_loader import ReviewsDataset, EssaysDataset
from captum.attr import IntegratedGradients, LimeBase
from IPython.display import HTML
import matplotlib
import numpy as np
from lime.lime_text import LimeTextExplainer
from torch import nn
import torch

def show_IG(model, dataset, ex_id=None, filter_params=None):
    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
    model.eval()
    if not filter_params:  filter_params = {}
    if not ex_id:  ex_id = dataset.filter_load(**filter_params, format='index')[0]
    ig = IntegratedGradients(model.forward_emb)
    example = dataset[ex_id][0].unsqueeze(dim=0).to(device)
    pred = model(example).argmax().cpu()
    
    # NOTE:  'baselines' should be zero tensor by default, which corresponds to having all <PAD> tokens.
    attributions = ig.attribute(inputs=model.get_embeddings(example), baselines=None, target=pred)
    scores = np.mean(attributions.detach().cpu().numpy(), axis=2).squeeze()

    encoding, label = dataset.__getitem__(ex_id, format='encoding')

    # create color mapping
    color_mapping = [   # RED => GREEN
        ((206, 35, 35), scores.min()),
        ((255, 255, 255), np.median(scores)),
        ((22, 206, 16), scores.max())
    ]
    
    # CREDIT:  https://databasecamp.de/en/ml/integrated-gradients-nlp  (reference for creating HTML display)
    def create_color_map(color_coords, color_bounds):
        def to_cmap_coord(x, level=0.0):  return( (level, np.interp(x, xp=[0,255], fp=[0,1]), np.interp(x, xp=[0,255], fp=[0,1])) )

        cmap_price_bounds = [np.interp(p, xp=[min(color_bounds), max(color_bounds)], fp=[0, 1]) for p in color_bounds]

        c_dict = {
            'red':tuple(to_cmap_coord(color_coords[i][0], cmap_price_bounds[i]) for i in range(len(color_coords))),
            'green':tuple(to_cmap_coord(color_coords[i][1], cmap_price_bounds[i]) for i in range(len(color_coords))),
            'blue':tuple(to_cmap_coord(color_coords[i][2], cmap_price_bounds[i]) for i in range(len(color_coords))),
        }
        
        return (matplotlib.colors.LinearSegmentedColormap('cmap', segmentdata=c_dict))
    c_map = create_color_map([c[0] for c in color_mapping], [c[1] for c in color_mapping])
    norm = matplotlib.colors.Normalize(vmin=scores.min(), vmax=scores.max())

    def build_html(text, c_map, norm, encoding, scores):
        def highlight(token, score):
            return f"<mark style=\"margin: 0; padding: 0; background-color:{matplotlib.colors.rgb2hex(c_map(norm(score)))}\">{token}</mark>"
        prev = (0, 0)
        cur_html = ""
        for i in range(len(encoding)):
            cur_html = cur_html + text[prev[1]: encoding.offsets[i][0]]
            cur_html = cur_html + highlight(encoding.tokens[i], scores[i])
            prev = encoding.offsets[i]
        return HTML(cur_html)
    
    #if dataset.score_type == 'categorical':  pred = pred + 1
    pred = pred.item()
    print("Example id:  ", ex_id)
    if 'category' in filter_params:
        print("Category:  ", filter_params['category'])
    print(f"{'Predicted Rating:' : <18}", pred if dataset.score_type == 'binary' else pred + 1)
    print(f"{'Actual Rating:' : <18}", label)

    return build_html(dataset.__getitem__(ex_id, format='raw')[0], c_map, norm, encoding, scores)

def show_LIME(model, dataset, num_samples=50, show_text=True, show_probs=True, ex_id=None, filter_params=None):
    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
    model.eval()
    if not filter_params:  filter_params = {}
    if not ex_id:  ex_id = dataset.filter_load(**filter_params, format='index')[0]
    pred = model(dataset[ex_id][0].unsqueeze(dim=0).to(device)).argmax().cpu().item()

    lime = LimeTextExplainer(class_names=list(sorted(dataset.get_unique_labels())))
    def model_prob_wrapper(dataset, model):
        def forward(text):
            print("TEXT: ", len(text))
            return nn.functional.softmax( model.forward(dataset.encode(text, mode='train').to(device)), dim=1 ).detach().cpu().numpy()
        return forward
    
    text, label = dataset.__getitem__(ex_id, format='raw')
    print("Example id:  ", ex_id)
    if 'category' in filter_params:
        print("Category:  ", filter_params['category'])
    print(f"{'Predicted Rating:' : <18}", pred if dataset.score_type == 'binary' else pred + 1)
    print(f"{'Actual Rating:' : <18}", label)
    explanation = lime.explain_instance(text, model_prob_wrapper(dataset, model), labels=(pred,), num_samples=num_samples)
    
    return explanation.show_in_notebook(text=show_text, labels=(pred,), predict_proba=show_probs)

### IG Experiments

* Not applicable to regression models (standardized scores)

In [89]:
SHORT_LEN = 1000
LARGE_LEN = [1000, 6000]
#  ------------  REVIEWS EXPERIMENTS  ------------  #

# Categorical, Short sequences, vocab_size ~ {500, 1000, 5000}
show_IG(load_model('reviews_cat_tk500_sq500'), ReviewsDataset(score_type='categorical', tokenizer='reviews_tokenizer_500', load_mode='lazy'), filter_params={'length': SHORT_LEN})
# show_IG(load_model('reviews_cat_tk1000_sq1000'), ReviewsDataset(score_type='categorical', tokenizer='reviews_tokenizer', load_mode='lazy'), filter_params={'length': SHORT_LEN})
# show_IG(load_model('reviews_cat_tk5000_sq1000'), ReviewsDataset(score_type='categorical', tokenizer='reviews_tokenizer_5000', load_mode='lazy'), filter_params={'length': SHORT_LEN})

# Binary, Short sequences, vocab_size ~ {500, 1000, 5000}
# show_IG(load_model('reviews_bin_tk500_sq500'), ReviewsDataset(score_type='binary', tokenizer='reviews_tokenizer_500', load_mode='lazy'), filter_params={'length': SHORT_LEN})
# show_IG(load_model('reviews_bin_tk1000_sq1000'), ReviewsDataset(score_type='binary', tokenizer='reviews_tokenizer', load_mode='lazy'), filter_params={'length': SHORT_LEN})
# show_IG(load_model('reviews_bin_tk5000_sq500'), ReviewsDataset(score_type='binary', tokenizer='reviews_tokenizer_5000', load_mode='lazy'), filter_params={'length': SHORT_LEN})

# Categorical, Longer sequences, vocab_size ~ {500, 1000, 5000}
# show_IG(load_model('reviews_cat_tk500_sq500'), ReviewsDataset(score_type='categorical', tokenizer='reviews_tokenizer_500', load_mode='lazy'), filter_params={'length': LARGE_LEN})
# show_IG(load_model('reviews_cat_tk1000_sq1000'), ReviewsDataset(score_type='categorical', tokenizer='reviews_tokenizer', load_mode='lazy'), filter_params={'length': LARGE_LEN})
# show_IG(load_model('reviews_cat_tk5000_sq1000'), ReviewsDataset(score_type='categorical', tokenizer='reviews_tokenizer_5000', load_mode='lazy'), filter_params={'length': LARGE_LEN})

# Binary, Longer sequences, vocab_size ~ {500, 1000, 5000}
# show_IG(load_model('reviews_bin_tk500_sq500'), ReviewsDataset(score_type='binary', tokenizer='reviews_tokenizer_500', load_mode='lazy'), filter_params={'length': LARGE_LEN})
# show_IG(load_model('reviews_bin_tk1000_sq1000'), ReviewsDataset(score_type='binary', tokenizer='reviews_tokenizer', load_mode='lazy'), filter_params={'length': LARGE_LEN})
# show_IG(load_model('reviews_bin_tk5000_sq500'), ReviewsDataset(score_type='binary', tokenizer='reviews_tokenizer_5000', load_mode='lazy'), filter_params={'length': LARGE_LEN})


#  ------------  ESSAYS EXPERIMENTS  ------------  #

# Categorical, Short sequences, vocab_size ~ {500, 1000, 5000}
# show_IG(load_model('essays_cat_tk500_sq1000'), EssaysDataset(score_type='categorical', tokenizer='essays_tokenizer_500', load_mode='lazy'), filter_params={'length': SHORT_LEN})
# show_IG(load_model('essays_cat_tk1000_sq500'), EssaysDataset(score_type='categorical', tokenizer='essays_tokenizer', load_mode='lazy'), filter_params={'length': SHORT_LEN})
# show_IG(load_model('essays_cat_tk5000_sq500'), EssaysDataset(score_type='categorical', tokenizer='essays_tokenizer_5000', load_mode='lazy'), filter_params={'length': SHORT_LEN})

# Binary, Short sequences, vocab_size ~ {500, 1000, 5000}
# show_IG(load_model('essays_bin_tk500_sq1000'), EssaysDataset(score_type='binary', tokenizer='essays_tokenizer_500', load_mode='lazy'), filter_params={'length': SHORT_LEN})
# show_IG(load_model('essays_bin_tk1000_sq1000'), EssaysDataset(score_type='binary', tokenizer='essays_tokenizer', load_mode='lazy'), filter_params={'length': SHORT_LEN})
# show_IG(load_model('essays_bin_tk5000_sq1000'), EssaysDataset(score_type='binary', tokenizer='essays_tokenizer_5000', load_mode='lazy'), filter_params={'length': SHORT_LEN})

# Categorical, Longer sequences, vocab_size ~ {500, 1000, 5000}
# show_IG(load_model('essays_cat_tk500_sq1000'), EssaysDataset(score_type='categorical', tokenizer='essays_tokenizer_500', load_mode='lazy'), filter_params={'length': LARGE_LEN})
# show_IG(load_model('essays_cat_tk1000_sq500'), EssaysDataset(score_type='categorical', tokenizer='essays_tokenizer', load_mode='lazy'), filter_params={'length': LARGE_LEN})
# show_IG(load_model('essays_cat_tk5000_sq500'), EssaysDataset(score_type='categorical', tokenizer='essays_tokenizer_5000', load_mode='lazy'), filter_params={'length': LARGE_LEN})

# Binary, Longer sequences, vocab_size ~ {500, 1000, 5000}
# show_IG(load_model('essays_bin_tk500_sq1000'), EssaysDataset(score_type='binary', tokenizer='essays_tokenizer_500', load_mode='lazy'), filter_params={'length': LARGE_LEN})
# show_IG(load_model('essays_bin_tk1000_sq1000'), EssaysDataset(score_type='binary', tokenizer='essays_tokenizer', load_mode='lazy'), filter_params={'length': LARGE_LEN})
# show_IG(load_model('essays_bin_tk5000_sq1000'), EssaysDataset(score_type='binary', tokenizer='essays_tokenizer_5000', load_mode='lazy'), filter_params={'length': LARGE_LEN})

Example id:   36148
Predicted Rating:  5
Actual Rating:     5


### LIME Experiments
* NOTE:  Should be applicable to regression models

In [ ]:
SHORT_LEN = 1000
LARGE_LEN = [1000, 6000]
#  ------------  REVIEWS EXPERIMENTS  ------------  #

# Categorical, Short sequences, vocab_size ~ {500, 1000, 5000}
show_LIME(load_model('reviews_cat_tk500_sq500'), ReviewsDataset(score_type='categorical', seq_len=500, tokenizer='reviews_tokenizer_500', load_mode='lazy'), filter_params={'length': SHORT_LEN})
# show_LIME(load_model('reviews_cat_tk1000_sq1000'), ReviewsDataset(score_type='categorical', seq_len=1000, tokenizer='reviews_tokenizer', load_mode='lazy'), filter_params={'length': SHORT_LEN})
# show_LIME(load_model('reviews_cat_tk5000_sq1000'), ReviewsDataset(score_type='categorical', seq_len=1000, tokenizer='reviews_tokenizer_5000', load_mode='lazy'), filter_params={'length': SHORT_LEN})

# Binary, Short sequences, vocab_size ~ {500, 1000, 5000}
# show_LIME(load_model('reviews_bin_tk500_sq500'), ReviewsDataset(score_type='binary', seq_len=500, tokenizer='reviews_tokenizer_500', load_mode='lazy'), filter_params={'length': SHORT_LEN})
# show_LIME(load_model('reviews_bin_tk1000_sq1000'), ReviewsDataset(score_type='binary', seq_len=1000, tokenizer='reviews_tokenizer', load_mode='lazy'), filter_params={'length': SHORT_LEN})
# show_LIME(load_model('reviews_bin_tk5000_sq500'), ReviewsDataset(score_type='binary', seq_len=500, tokenizer='reviews_tokenizer_5000', load_mode='lazy'), filter_params={'length': SHORT_LEN})

# Categorical, Longer sequences, vocab_size ~ {500, 1000, 5000}
# show_LIME(load_model('reviews_cat_tk500_sq500'), ReviewsDataset(score_type='categorical', seq_len=500, tokenizer='reviews_tokenizer_500', load_mode='lazy'), filter_params={'length': LARGE_LEN})
# show_LIME(load_model('reviews_cat_tk1000_sq1000'), ReviewsDataset(score_type='categorical', seq_len=1000, tokenizer='reviews_tokenizer', load_mode='lazy'), filter_params={'length': LARGE_LEN})
# show_LIME(load_model('reviews_cat_tk5000_sq1000'), ReviewsDataset(score_type='categorical', seq_len=1000, tokenizer='reviews_tokenizer_5000', load_mode='lazy'), filter_params={'length': LARGE_LEN})

# Binary, Longer sequences, vocab_size ~ {500, 1000, 5000}
# show_LIME(load_model('reviews_bin_tk500_sq500'), ReviewsDataset(score_type='binary', seq_len=500, tokenizer='reviews_tokenizer_500', load_mode='lazy'), filter_params={'length': LARGE_LEN})
# show_LIME(load_model('reviews_bin_tk1000_sq1000'), ReviewsDataset(score_type='binary', seq_len=1000, tokenizer='reviews_tokenizer', load_mode='lazy'), filter_params={'length': LARGE_LEN})
# show_LIME(load_model('reviews_bin_tk5000_sq500'), ReviewsDataset(score_type='binary', seq_len=500, tokenizer='reviews_tokenizer_5000', load_mode='lazy'), filter_params={'length': LARGE_LEN})


#  ------------  ESSAYS EXPERIMENTS  ------------  #

# Categorical, Short sequences, vocab_size ~ {500, 1000, 5000}
# show_LIME(load_model('essays_cat_tk500_sq1000'), EssaysDataset(score_type='categorical', seq_len=1000, tokenizer='essays_tokenizer_500', load_mode='lazy'), filter_params={'length': SHORT_LEN})
# show_LIME(load_model('essays_cat_tk1000_sq500'), EssaysDataset(score_type='categorical', seq_len=500, tokenizer='essays_tokenizer', load_mode='lazy'), filter_params={'length': SHORT_LEN})
# show_LIME(load_model('essays_cat_tk5000_sq500'), EssaysDataset(score_type='categorical', seq_len=500, tokenizer='essays_tokenizer_5000', load_mode='lazy'), filter_params={'length': SHORT_LEN})

# Binary, Short sequences, vocab_size ~ {500, 1000, 5000}
# show_LIME(load_model('essays_bin_tk500_sq500'), EssaysDataset(score_type='binary', seq_len=500, tokenizer='essays_tokenizer_500', load_mode='lazy'), filter_params={'length': SHORT_LEN})
# show_LIME(load_model('essays_bin_tk1000_sq500'), EssaysDataset(score_type='binary', seq_len=500, tokenizer='essays_tokenizer', load_mode='lazy'), filter_params={'length': SHORT_LEN})
# show_LIME(load_model('essays_bin_tk5000_sq500'), EssaysDataset(score_type='binary', seq_len=500, tokenizer='essays_tokenizer_5000', load_mode='lazy'), filter_params={'length': SHORT_LEN})

# Categorical, Longer sequences, vocab_size ~ {500, 1000, 5000}
# show_LIME(load_model('essays_cat_tk500_sq1000'), EssaysDataset(score_type='categorical', seq_len=1000, tokenizer='essays_tokenizer_500', load_mode='lazy'), filter_params={'length': LARGE_LEN})
# show_LIME(load_model('essays_cat_tk1000_sq500'), EssaysDataset(score_type='categorical', seq_len=500, tokenizer='essays_tokenizer', load_mode='lazy'), filter_params={'length': LARGE_LEN})
# show_LIME(load_model('essays_cat_tk5000_sq500'), EssaysDataset(score_type='categorical', seq_len=500, tokenizer='essays_tokenizer_5000', load_mode='lazy'), filter_params={'length': LARGE_LEN})

# Binary, Longer sequences, vocab_size ~ {500, 1000, 5000}
# show_LIME(load_model('essays_bin_tk500_sq500'), EssaysDataset(score_type='binary', seq_len=500, tokenizer='essays_tokenizer_500', load_mode='lazy'), filter_params={'length': LARGE_LEN})
# show_LIME(load_model('essays_bin_tk1000_sq500'), EssaysDataset(score_type='binary', seq_len=500, tokenizer='essays_tokenizer', load_mode='lazy'), filter_params={'length': LARGE_LEN})
# show_LIME(load_model('essays_bin_tk5000_sq500'), EssaysDataset(score_type='binary', seq_len=500, tokenizer='essays_tokenizer_5000', load_mode='lazy'), filter_params={'length': LARGE_LEN})


In [ ]:
# show_LIME(load_model('essays_trans_cat'), EssaysDataset(score_type='categorical', seq_len=500, load_mode='lazy'),
#           num_samples=500, ex_id=290)
# show_LIME(load_model('reviews_trans_cat'), ReviewsDataset(score_type='categorical', seq_len=500, load_mode='lazy'),
#           num_samples=500, ex_id=1301)
# show_LIME(load_model('essays_dan_bin'), ReviewsDataset(score_type='binary', seq_len=500, load_mode='lazy'),
#           num_samples=500, filter_params={'category': 'appliances', 'length': 800, 'score': 0})